In [43]:
import pandas as pd

df = pd.read_csv("cleaned_2023.csv")
df = df.fillna('NULL')

In [44]:
player = df[['player_id', 'ustat_id', 'player_name_futbin',
             'player_name_ustat', 'dob', 'nation', 'pos']]

team = df[['club_id', 'team_title_futbin', 'team_title_ustat']].drop_duplicates()
team = team[team['team_title_futbin'] != '\n                                           Controlled                                       ']

# competition = df

player_stats = df[['player_id', 'year', 'league_id', 'club_id', 'games',
                   'time', 'goals', 'assists', 'xG', 'xA']]

# team_stats = df[['team_id',

# comp_stats = df

player_ratings = df[['player_id', 'year', 'rating', 'pac', 'acceleration', 'sprint_speed', 'sho', 'positioning', 
                     'finishing', 'shot_power', 'long_shots', 'volleys', 'penalties', 'pas', 'vision', 'crossing', 
                     'fk_accuracy', 'short_passing', 'long_passing', 'curve', 'dri', 'agility', 'balance',
                     'reactions', 'ball_control', 'dribbling', 'composure', 'def', 'interceptions', 'heading_acc', 
                     'def_awareness', 'standing_tackle', 'sliding_tackle', 'phy', 'jumping', 'stamina', 
                     'strength', 'aggression', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 
                     'gk_speed', 'gk_positioning']]

In [45]:
team

,club_id,team_title_futbin,team_title_ustat
0,1943,AFC Bournemouth,Bournemouth
1,17,Southampton,Southampton
2,1799,Crystal Palace,Crystal Palace
3,144,Fulham,Fulham
4,13,Newcastle Utd,Newcastle United
...,...,...,...
1128,166,Hertha Berlin,Hertha Berlin
1136,23,M'gladbach,Borussia M.Gladbach
1137,36,VfB Stuttgart,VfB Stuttgart
1141,175,VfL Wolfsburg,Wolfsburg


In [32]:
!pip install psycopg2

In [26]:
import pyscopg2

conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="password")

dfs = [player, team, player_stats, player_ratings]
tables = ['player', 'team', 'player_stats', 'player_ratings']

for i in range(len(dfs)):
    dfs[i].to_sql(tables[i], con=conn, if_exists='append')

ModuleNotFoundError: No module named 'pyscopg2'

In [48]:
def get_inserts(df, table, file):
    for index, row in df.iterrows():       
        file.write('INSERT INTO '+table+' VALUES '+ str(tuple(row.values))+'\n')

dfs = [player, team, player_stats, player_ratings]
tables = ['player', 'team', 'player_stats', 'player_ratings']

f = open(os.path.join(os.path.abspath(''), "inserts.sql"), "w+")

for i in range(len(dfs)):
    get_inserts(dfs[i], tables[i], f);
    
f.close()

In [39]:
df.isna().sum().sum()

11371